In [2]:
import numpy as np
from scipy.spatial.distance import cdist


In [3]:
np.random.choice(1000,3,replace=False)

array([307, 671, 233])

In [4]:
test = np.random.choice(10,(20,3))
print(test)
centroids = test[np.random.choice(20,5)]
print("\n", centroids)

[[5 6 8]
 [7 5 3]
 [1 3 8]
 [2 1 4]
 [9 7 6]
 [1 1 1]
 [0 9 3]
 [3 5 5]
 [6 2 1]
 [9 2 3]
 [8 6 7]
 [8 8 1]
 [2 8 0]
 [5 1 5]
 [4 4 7]
 [8 4 6]
 [4 1 3]
 [0 3 2]
 [8 0 4]
 [9 8 8]]

 [[8 6 7]
 [4 1 3]
 [1 1 1]
 [5 1 5]
 [0 3 2]]


In [6]:
cdist(test, centroids)

array([[ 3.16227766,  7.14142843,  9.48683298,  5.83095189,  8.36660027],
       [ 4.24264069,  5.        ,  7.48331477,  4.89897949,  7.34846923],
       [ 7.68114575,  6.164414  ,  7.28010989,  5.38516481,  6.08276253],
       [ 8.36660027,  2.23606798,  3.16227766,  3.16227766,  3.46410162],
       [ 1.73205081,  8.36660027, 11.18033989,  7.28010989, 10.63014581],
       [10.48808848,  3.60555128,  0.        ,  5.65685425,  2.44948974],
       [ 9.43398113,  8.94427191,  8.30662386,  9.64365076,  6.08276253],
       [ 5.47722558,  4.58257569,  6.        ,  4.47213595,  4.69041576],
       [ 7.48331477,  3.        ,  5.09901951,  4.24264069,  6.164414  ],
       [ 5.74456265,  5.09901951,  8.30662386,  4.58257569,  9.11043358],
       [ 0.        ,  7.54983444, 10.48808848,  6.164414  ,  9.89949494],
       [ 6.32455532,  8.30662386,  9.89949494,  8.60232527,  9.48683298],
       [ 9.43398113,  7.87400787,  7.14142843,  9.11043358,  5.74456265],
       [ 6.164414  ,  2.23606798,  5.6

In [7]:
np.argmin(cdist(test, centroids, metric="euclidean"), axis=1)

array([0, 0, 3, 1, 0, 2, 4, 3, 1, 3, 0, 0, 4, 3, 3, 0, 1, 4, 3, 0])

In [8]:
np.inf

inf

In [9]:
np.zeros((3,3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [10]:
np.sum(np.square(test - centroids[0]))

876

In [106]:
import numpy as np
from scipy.spatial.distance import cdist


class KMeans:
    def __init__(self, k: int, tol: float = 1e-6, max_iter: int = 100):
        """
        In this method you should initialize whatever attributes will be required for the class.

        You can also do some basic error handling.

        What should happen if the user provides the wrong input or wrong type of input for the
        argument k?

        inputs:
            k: int
                the number of centroids to use in cluster fitting
            tol: float
                the minimum error tolerance from previous error during optimization to quit the model fit
            max_iter: int
                the maximum number of iterations before quitting model fit
        """

        if k <= 0: raise Exception("k must be a positive integer") #TypeError

        self.k = k
        self.tol = tol
        self.max_iter = max_iter

    def fit(self, mat: np.ndarray):
        """
        Fits the kmeans algorithm onto a provided 2D matrix.
        As a bit of background, this method should not return anything.
        The intent here is to have this method find the k cluster centers from the data
        with the tolerance, then you will use .predict() to identify the
        clusters that best match some data that is provided.

        In sklearn there is also a fit_predict() method that combines these
        functions, but for now we will have you implement them both separately.

        inputs:
            mat: np.ndarray
                A 2D matrix where the rows are observations and columns are features
        """

        self.mat = mat
        self.observations, self.features = self.mat.shape

        if self.observations < self.k: raise Exception("Cannot assign " + str(self.observations) + " observations to " + str(self.k) + " clusters")

        #self.centroids = mat[np.random.choice(self.observations, self.k, replace=False)]
        self.centroids = self._init_centroids(mat)
        self.pred_labels = np.zeros((self.observations, 1))
     
        i = 0
        error = np.inf

        while i < self.max_iter and error > self.tol:
            self.pred_labels = self.predict(self.mat)
            #print("pred: ",self.pred_labels)
            #print("pred labels: ", self.pred_labels)
            self.old = self.centroids
            self.centroids = self.get_centroids()
            print("old: ", self.old)
            print("centroids: ", self.centroids)
            #print("centroids: ", self.centroids)
            #if i == 0: error = self.get_error()
            #else: error = error - self.get_error()
            error = self.get_error()
            print(error)
            i += 1
        #self.pred_labels = self.predict(self.mat)


    def predict(self, mat: np.ndarray) -> np.ndarray:
        """
        Predicts the cluster labels for a provided matrix of data points--
            question: what sorts of data inputs here would prevent the code from running?
            How would you catch these sorts of end-user related errors?
            What if, for example, the matrix is of a different number of features than
            the data that the clusters were fit on?

        inputs:
            mat: np.ndarray
                A 2D matrix where the rows are observations and columns are features

        outputs:
            np.ndarray
                a 1D array with the cluster label for each of the observations in `mat`
        """
        
        return np.argmin(cdist(self.mat, self.centroids, metric="euclidean"), axis=1)


    def get_error(self) -> float:
        """
        Returns the final squared-mean error of the fit model. You can either do this by storing the
        original dataset or recording it following the end of model fitting.

        outputs:
            float
                the squared-mean error of the fit model
        """
        #mse = np.zeros(self.k)
        mse = np.sum(np.square(self.old - self.centroids))

        #for cluster in range(self.k):
            #print("pred: ", self.mat[cluster == self.pred_labels])
            #print("centroids: ",self.centroids[cluster])
            #mse[cluster] = np.sum(np.square(self.mat[cluster == self.pred_labels] - self.centroids[cluster]))
            #mse[cluster] = np.sum(np.square(self.old - self.centroids))
        return float(np.sum(mse))

    def get_centroids(self) -> np.ndarray:
        """
        Returns the centroid locations of the fit model.

        outputs:
            np.ndarray
                a `k x m` 2D matrix representing the cluster centroids of the fit model
        """

        fit_centroids = np.zeros((self.k, self.features))

        for cluster in range(self.k):
            #print("centroids fit: ", self.mat[cluster == self.pred_labels, :])
            
            fit_centroids[cluster, :] = np.mean(self.mat[cluster == self.pred_labels, :], axis = 0)
            
        return fit_centroids
    
    def _init_centroids(self, mat:np.ndarray) -> np.ndarray:

        centroids = mat[np.random.choice(self.observations, self.k)]
        mod_mat = mat
        # replacing each random centroid

        for centroid in range(self.k - 1):
            ind = np.where((mod_mat == centroids[centroid]).all(axis=1))
            print("centroids: ", centroids[centroid])
            print("mod mat: ", mod_mat)
            print("ind: ", ind)
            mod_mat = np.delete(mod_mat, ind, axis=0)
            print("mod mat: ", mod_mat)
            print("c: ", centroids[centroid])
            
            centroids[centroid + 1] = mat[np.argmax(np.sum(np.square(centroids[centroid] - mod_mat), axis=1))]

        return centroids




In [107]:
new = KMeans(3)
new.fit(test)
#print(test)

#np.where((test==[7,4,5]).all(axis=1))
#(vals == (0, 1)).all(axis=1)

centroids:  [3 0 4]
mod mat:  [[8 9 1]
 [5 8 0]
 [7 3 8]
 [9 1 4]
 [0 7 9]
 [7 4 5]
 [0 4 8]
 [3 0 7]
 [5 5 5]
 [1 8 5]
 [3 0 4]
 [2 1 5]
 [3 4 7]
 [3 5 3]
 [3 2 2]
 [8 3 1]
 [5 0 3]
 [1 3 5]
 [0 4 5]
 [0 5 6]]
ind:  (array([10]),)
mod mat:  [[8 9 1]
 [5 8 0]
 [7 3 8]
 [9 1 4]
 [0 7 9]
 [7 4 5]
 [0 4 8]
 [3 0 7]
 [5 5 5]
 [1 8 5]
 [2 1 5]
 [3 4 7]
 [3 5 3]
 [3 2 2]
 [8 3 1]
 [5 0 3]
 [1 3 5]
 [0 4 5]
 [0 5 6]]
c:  [3 0 4]
centroids:  [8 9 1]
mod mat:  [[8 9 1]
 [5 8 0]
 [7 3 8]
 [9 1 4]
 [0 7 9]
 [7 4 5]
 [0 4 8]
 [3 0 7]
 [5 5 5]
 [1 8 5]
 [2 1 5]
 [3 4 7]
 [3 5 3]
 [3 2 2]
 [8 3 1]
 [5 0 3]
 [1 3 5]
 [0 4 5]
 [0 5 6]]
ind:  (array([0]),)
mod mat:  [[5 8 0]
 [7 3 8]
 [9 1 4]
 [0 7 9]
 [7 4 5]
 [0 4 8]
 [3 0 7]
 [5 5 5]
 [1 8 5]
 [2 1 5]
 [3 4 7]
 [3 5 3]
 [3 2 2]
 [8 3 1]
 [5 0 3]
 [1 3 5]
 [0 4 5]
 [0 5 6]]
c:  [8 9 1]
old:  [[3 0 4]
 [8 9 1]
 [0 4 8]]
centroids:  [[4.7        2.1        4.6       ]
 [7.         6.66666667 0.66666667]
 [0.71428571 5.         6.42857143]]
18.195147392

In [13]:
import numpy as np
'''
from cluster import (
        KMeans, 
        Silhouette, 
        make_clusters,
        plot_clusters,
        plot_multipanel)
'''


def main():

    # create tight clusters
    clusters, labels = make_clusters(scale=0.3)
    plot_clusters(clusters, labels, filename="figures/tight_clusters.png")

    # create loose clusters
    clusters, labels = make_clusters(scale=2)
    plot_clusters(clusters, labels, filename="figures/loose_clusters.png")

    """
    uncomment this section once you are ready to visualize your kmeans + silhouette implementation
    """
    clusters, labels = make_clusters(k=4, scale=1)
    km = KMeans(k=4)
    km.fit(clusters)
    pred = km.predict(clusters)
    #scores = Silhouette().score(clusters, pred)
    plot_multipanel(clusters, labels, pred, scores)
    

if __name__ == "__main__":
    main()


NameError: name 'make_clusters' is not defined

In [14]:
a = [2,5,4]
cdist(a,test,metric="euclidean")

ValueError: XA must be a 2-dimensional array.

In [ ]:
#print(test)
dists = cdist(test, test, metric="euclidean")
np.sum(dists, axis=1)/(dists.shape[0] - 1)
#== np.mean(dists, axis=1)
#dists
#print(dists, "\n\n\n\n\n")
#dists.ravel()[::dists.shape[1]+1] = dists.max()+1
#print(dists)
#print(test[dists.argmin(1)])
#np.triu(dists)

In [15]:
import numpy as np
from scipy.spatial.distance import cdist


class Silhouette:
    def __init__(self):
        """
        inputs:
            none
        """

    def score(self, X: np.ndarray, y: np.ndarray) -> np.ndarray:
        """
        calculates the silhouette score for each of the observations

        inputs:
            X: np.ndarray
                A 2D matrix where the rows are observations and columns are features.

            y: np.ndarray
                a 1D array representing the cluster labels for each of the observations in `X`

        outputs:
            np.ndarray
                a 1D array with the silhouette scores for each of the observations in `X`
        """

        nclusters = np.amax(y) + 1
        nobs, nlabels = X.shape
        labels = np.arange(0, np.amax(y) + 1)
        a = np.zeros(nobs)

        for cluster in labels:
        #for cluster in np.unique(labels):            
            print("something")
            cluster_type = X[cluster == y]
            inds = np.where(cluster == y)
            dists = cdist(cluster_type, cluster_type, metric="euclidean")
            
            a[inds] = np.sum(dists, axis=1) / (dists.shape[0] - 1)

        b = np.zeros(nobs)

        for cluster in labels:
        #for cluster in np.unique(labels):
            print("times")
            cluster_type_og = X[cluster == y]
            inds = np.where(cluster == y)
            #other_labels = np.delete(labels, inds)
            other_labels = np.delete(y, inds)
            #print("shape of ")
            min_dists = np.full(X[cluster == y].shape[0], np.inf)
            #print("shape: ", other_labels)
            #print("num in this cluster: ", cluster)
            
            #for other_cluster in range(len(other_labels)):
            for other_cluster in np.unique(other_labels):
                print("others")
                #print("num in others : ", other_cluster)
                #print("other labels: ", other_labels)
                cluster_type = X[other_cluster == y]
                #print("other cluster type: ", cluster_type)

                #dists = cdist(cluster_type, cluster_type, metric="euclidean")
                #print("shape of og: ", cluster_type_og.shape)
                #print("shape of later: ", cluster_type.shape)
                dists = cdist(cluster_type_og, cluster_type, metric="euclidean")
                #print("dists: ", dists)
                avg_dists_per_cluster = np.mean(dists, axis=1)
                min_dists = np.minimum(avg_dists_per_cluster, min_dists)
                # make sure that the orientation of the matrix is correct
            
            b[inds] = min_dists
            
        #print("b: ", b)
        #print("a: ", a)
        return (b - a) / (np.maximum(a, b))
    
    



'\ns = Silhouette()\n\nlabels=np.array([2, 0, 1, 0, 1, 1, 2, 1, 2, 1, 1, 0, 2, 2, 0, 0, 0, 1, 1, 2])\nmean = np.mean(test, axis = 1)\nprint("mean before: ", mean)\nind = np.where(labels<1)\nprint("ind: ",ind)\nmean[ind] = np.arange(400,406)\nprint("mean after: ", mean)\n\n#s.score(test, labels)\n'

In [17]:
s = Silhouette()
test = np.random.choice(10,(20,3))
#print("test: ", test)
labels=np.array([2, 0, 1, 0, 1, 1, 2, 1, 2, 1, 1, 0, 2, 2, 0, 0, 0, 1, 1, 2])
s.score(test,labels)

something
something
something
times
others
others
times
others
others
times
others
others


array([-0.22799814, -0.02182546,  0.05299444,  0.21327907, -0.13102717,
       -0.17945865, -0.02839862,  0.06136987, -0.12520589, -0.16049024,
       -0.1940939 , -0.25599731, -0.12126453, -0.0617686 ,  0.13523879,
        0.33251578,  0.21746058, -0.10120723, -0.16300576, -0.02863051])

In [18]:
c=np.arange(0,14)
c

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [19]:
min_dists = np.full(test.shape[0], np.inf)
min_dists

array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf])

In [20]:
x=[0,1,2,3,4]
np.amax(x)

4

In [21]:
x>2

TypeError: '>' not supported between instances of 'list' and 'int'

In [81]:
for i in labels: print(i)

2
0
1
0
1
1
2
1
2
1
1
0
2
2
0
0
0
1
1
2


In [240]:
p = [4, 4, 2, 3, 8, 9]

In [147]:
np.mean(test, axis=1)

array([6.66666667, 6.33333333, 2.        , 2.        , 4.        ,
       5.66666667, 6.66666667, 4.66666667, 4.        , 6.66666667,
       3.66666667, 5.        , 6.66666667, 5.        , 3.66666667,
       6.        , 7.66666667, 2.        , 2.        , 5.33333333])

In [23]:
100- test

array([[ 92,  91,  99],
       [ 95,  92, 100],
       [ 93,  97,  92],
       [ 91,  99,  96],
       [100,  93,  91],
       [ 93,  96,  95],
       [100,  96,  92],
       [ 97, 100,  93],
       [100,  99,  93],
       [ 99,  92,  95],
       [ 97, 100,  96],
       [ 98,  99,  95],
       [ 97,  96,  93],
       [ 97,  95,  97],
       [ 97,  98,  98],
       [ 92,  97,  99],
       [ 95, 100,  97],
       [ 99,  97,  95],
       [100,  96,  95],
       [100,  95,  94]])

In [27]:
centroids = test[np.random.choice(20, 1)]
centroids

array([[9, 1, 4]])

In [28]:
test

array([[8, 9, 1],
       [5, 8, 0],
       [7, 3, 8],
       [9, 1, 4],
       [0, 7, 9],
       [7, 4, 5],
       [0, 4, 8],
       [3, 0, 7],
       [0, 1, 7],
       [1, 8, 5],
       [3, 0, 4],
       [2, 1, 5],
       [3, 4, 7],
       [3, 5, 3],
       [3, 2, 2],
       [8, 3, 1],
       [5, 0, 3],
       [1, 3, 5],
       [0, 4, 5],
       [0, 5, 6]])

In [35]:
t = [3,4,5]
np.sum(np.square(t-test), axis =1)

array([66, 45, 26, 46, 34, 16, 18, 20, 22, 20, 17, 10,  4,  5, 13, 42, 24,
        5,  9, 11])

In [44]:
new = np.amin(np.sum(np.square(t-test), axis =1))
new

4

In [110]:
test[0,1]

9

In [456]:
import numpy as np
import heapq
from typing import Union

class Graph:

    def __init__(self, adjacency_mat: Union[np.ndarray, str]):
        """
    
        Unlike the BFS assignment, this Graph class takes an adjacency matrix as input. `adjacency_mat` 
        can either be a 2D numpy array of floats or a path to a CSV file containing a 2D numpy array of floats.

        In this project, we will assume `adjacency_mat` corresponds to the adjacency matrix of an undirected graph.
    
        """
        if type(adjacency_mat) == str:
            self.adj_mat = self._load_adjacency_matrix_from_csv(adjacency_mat)
        elif type(adjacency_mat) == np.ndarray:
            self.adj_mat = adjacency_mat
        else: 
            raise TypeError('Input must be a valid path or an adjacency matrix')
        self.mst = None

    def _load_adjacency_matrix_from_csv(self, path: str) -> np.ndarray:
        with open(path) as f:
            return np.loadtxt(f, delimiter=',')

    def construct_mst(self):
        """
    
        TODO: Given `self.adj_mat`, the adjacency matrix of a connected undirected graph, implement Prim's 
        algorithm to construct an adjacency matrix encoding the minimum spanning tree of `self.adj_mat`. 
            
        `self.adj_mat` is a 2D numpy array of floats. Note that because we assume our input graph is
        undirected, `self.adj_mat` is symmetric. Row i and column j represents the edge weight between
        vertex i and vertex j. An edge weight of zero indicates that no edge exists. 
        
        This function does not return anything. Instead, store the adjacency matrix representation
        of the minimum spanning tree of `self.adj_mat` in `self.mst`. We highly encourage the
        use of priority queues in your implementation. Refer to the heapq module, particularly the 
        `heapify`, `heappop`, and `heappush` functions.

        """
        #self.mst = None
        self.mst = np.zeros(self.adj_mat.shape)

        visited = []
        visited.append(0)
        h = []
        row = self.adj_mat[0,:]
        row_ind = 0
        insert = row[np.nonzero(row)]
        
        for i in insert: 
            heapq.heappush(h, i)
        
        while len(visited) < self.adj_mat.shape[0]:
            lowest = heapq.heappop(h)
            print("row: ", row)
            dest = np.where(row == lowest)[0][0]
            # just take the first vertex w that edge weight
            if dest not in visited:
                self.mst[row_ind,dest] = lowest
                row_ind = dest
                visited.append(dest)
                
                row = self.adj_mat[dest,:]
                insert = row[np.nonzero(row)]
                for i in insert: 
                    heapq.heappush(h, i)
            else: 
                row = self.adj_mat[np.where(self.adj_mat == lowest)[0][0]]
                row_ind = np.where(self.adj_mat == lowest)[0][0]
                
            if (not h) and (len(visited) < self.adj_mat.shape[0]): raise Exception("The input graph is disconnected")
        print(self.mst)


In [457]:
g = Graph("/Users/brysonchoy/Documents/bmi203/hw4-prim-mst/data/small.csv")

In [458]:
g.construct_mst()

row:  [0. 5. 0. 5.]
row:  [5. 0. 1. 2.]
row:  [0. 1. 0. 4.]
row:  [5. 0. 1. 2.]
[[0. 5. 0. 0.]
 [0. 0. 1. 2.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [320]:
p = np.arange(0,20)
print(p)
np.delete(p,4)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


array([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])

In [381]:
test[np.where(test==4)[0][0]]

array([9, 1, 4])

In [366]:
test

array([[8, 9, 1],
       [5, 8, 0],
       [7, 3, 8],
       [9, 1, 4],
       [0, 7, 9],
       [7, 4, 5],
       [0, 4, 8],
       [3, 0, 7],
       [5, 5, 5],
       [1, 8, 5],
       [3, 0, 4],
       [2, 1, 5],
       [3, 4, 7],
       [3, 5, 3],
       [3, 2, 2],
       [8, 3, 1],
       [5, 0, 3],
       [1, 3, 5],
       [0, 4, 5],
       [0, 5, 6]])

In [393]:
np.zeros((test.shape))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [402]:
test[np.nonzero(test)]

array([8, 9, 1, 5, 8, 7, 3, 8, 9, 1, 4, 7, 9, 7, 4, 5, 4, 8, 3, 7, 5, 5,
       5, 1, 8, 5, 3, 4, 2, 1, 5, 3, 4, 7, 3, 5, 3, 3, 2, 2, 8, 3, 1, 5,
       3, 1, 3, 5, 4, 5, 5, 6])

In [403]:
emp = []

In [406]:
if not emp: print("stop")

stop


In [447]:
bad_adjmat = np.arange(-35, -3, 2)
bad_adjmat = np.reshape(bad_adjmat,(4,4))

In [448]:
bad_adjmat

array([[-35, -33, -31, -29],
       [-27, -25, -23, -21],
       [-19, -17, -15, -13],
       [-11,  -9,  -7,  -5]])

In [426]:
np.any(bad_adjmat.diagonal()) == 3

False

In [441]:
y = np.asarray([[0, 5, 0, 0],
 [0, 0, 1, 2],
 [0, 0, 0, 0],
 [0, 0, 0, 0]])

In [446]:
if np.any(y.diagonal()): print("bad!")

In [449]:
test[0][0] = -5
test

array([[-5,  9,  1],
       [ 5,  8,  0],
       [ 7,  3,  8],
       [ 9,  1,  4],
       [ 0,  7,  9],
       [ 7,  4,  5],
       [ 0,  4,  8],
       [ 3,  0,  7],
       [ 5,  5,  5],
       [ 1,  8,  5],
       [ 3,  0,  4],
       [ 2,  1,  5],
       [ 3,  4,  7],
       [ 3,  5,  3],
       [ 3,  2,  2],
       [ 8,  3,  1],
       [ 5,  0,  3],
       [ 1,  3,  5],
       [ 0,  4,  5],
       [ 0,  5,  6]])

In [452]:
np.any(y < 0)

False

In [459]:
test.T

array([[-5,  5,  7,  9,  0,  7,  0,  3,  5,  1,  3,  2,  3,  3,  3,  8,
         5,  1,  0,  0],
       [ 9,  8,  3,  1,  7,  4,  4,  0,  5,  8,  0,  1,  4,  5,  2,  3,
         0,  3,  4,  5],
       [ 1,  0,  8,  4,  9,  5,  8,  7,  5,  5,  4,  5,  7,  3,  2,  1,
         3,  5,  5,  6]])

In [462]:
new = np.asarray([[0, 5, 0, 5],
[5, 0, 1, 2],
[0, 1, 0, 4],
[5, 2, 4, 0]])


In [470]:
np.any(test == test.T)

/var/folders/bq/wgkng3ln1q908g_9zwmkkq480000gp/T/ipykernel_33070/237035954.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.any(test == test.T)


False

In [471]:
test.shape == new.shape

False

In [474]:
x=1e+6

In [475]:
x

1000000.0